<a href="https://colab.research.google.com/github/Vaalentinaa/data_engineering_challenge/blob/main/Data_Engineering_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=5c3fac1618afb90c8ba92d14717f9d4714355e5d05f2316c2eec9caf9058c112
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!unzip '/content/drive/MyDrive/data (1).zip'

replace ticket_line.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [11]:
stores = spark.read.csv('stores.csv', header=True, sep=",")

In [12]:
stores.show()

+--------+-------+-------+
|store_id|country|version|
+--------+-------+-------+
|      40|     DE|      1|
|      41|     DE|      1|
|      42|     DE|      1|
|      43|     DE|      1|
|      44|     DE|      1|
|      45|     DE|      1|
+--------+-------+-------+



In [13]:
products = spark.read.json('products.json')

In [14]:
products.show()

+--------------------+----------+--------------------+
|          categories|product_id|        product_name|
+--------------------+----------+--------------------+
|[{1023, Dairy pro...|         1|        Milbona milk|
|[{1023, Dairy pro...|         2|      Milbona cheese|
|[{1023, Dairy pro...|         3|       Milbona yogur|
|    [{1025, Cereal}]|         4|    Crownfield black|
|    [{1025, Cereal}]|         5|Crownfield with milk|
|   [{1026, Cookies}]|         6|      Sondey cookeys|
|[{1026, Cookies},...|         7|    Sondey digestive|
|   [{1026, Cookies}]|         8|    Sondey hazelnuts|
|[{1028, Vegetables}]|         9|      Freshona beans|
|[{1028, Vegetables}]|        10|   Freshona potatoes|
+--------------------+----------+--------------------+



In [67]:
from pyspark.sql.functions import explode

In [77]:
from pyspark.sql.functions import col

In [68]:
products_transformed = products.select("product_id","product_name",explode("categories").category_name)

In [85]:
products = products_transformed.select("product_id","product_name",col("col.category_name").alias("category_name"),col("col.category_id").alias("category_id"))

In [87]:
products.show()

+----------+--------------------+-------------+-----------+
|product_id|        product_name|category_name|category_id|
+----------+--------------------+-------------+-----------+
|         1|        Milbona milk|Dairy product|       1023|
|         2|      Milbona cheese|Dairy product|       1023|
|         3|       Milbona yogur|Dairy product|       1023|
|         4|    Crownfield black|       Cereal|       1025|
|         5|Crownfield with milk|       Cereal|       1025|
|         6|      Sondey cookeys|      Cookies|       1026|
|         7|    Sondey digestive|      Cookies|       1026|
|         7|    Sondey digestive|       Cereal|       1025|
|         8|    Sondey hazelnuts|      Cookies|       1026|
|         9|      Freshona beans|   Vegetables|       1028|
|        10|   Freshona potatoes|   Vegetables|       1028|
+----------+--------------------+-------------+-----------+



In [15]:
tickets = spark.read.csv('ticket_line.csv',header= True, sep=',')

In [16]:
tickets.show()

+---------+----------+--------+----------+--------+
|ticket_id|product_id|store_id|      date|quantity|
+---------+----------+--------+----------+--------+
|       22|         8|      41|2022-01-02|       3|
|      258|         6|      40|2022-01-02|       2|
|      623|         2|      45|2022-01-02|       2|
|       37|         5|      47|2022-01-02|       2|
|      378|         4|      47|2022-01-02|       2|
|      391|         1|      46|2022-01-02|       2|
|      399|        10|      47|2022-01-02|       3|
|      727|         3|      45|2022-01-02|       3|
|      994|         2|      44|2022-01-02|       2|
|      844|         6|      42|2022-01-02|       3|
|      234|        10|      42|2022-01-02|       1|
|      206|         7|      44|2022-01-02|       2|
|      477|        10|      44|2022-01-02|       3|
|      496|         8|      43|2022-01-02|       3|
|      586|         1|      47|2022-01-02|       3|
|      511|        10|      41|2022-01-02|       3|
|      507| 

# 1. Find how many different stores is each product being sold. Please consider only the stores provided in the store.csv file, as not all stores are included in the Lidl Plus program.





Answer using SQL

In [26]:
tickets.createOrReplaceTempView("EMP")
stores.createOrReplaceTempView("STORE")
spark.sql("SELECT product_id, count(DISTINCT store_id) as count FROM EMP WHERE store_id IN (SELECT store_id FROM STORE) GROUP BY product_id").show()

+----------+-----+
|product_id|count|
+----------+-----+
|         7|    6|
|         3|    6|
|         8|    6|
|         5|    6|
|         6|    6|
|         9|    6|
|         1|    6|
|        10|    6|
|         4|    6|
|         2|    6|
+----------+-----+



Answer without SQL


In [36]:
tickets.join(stores, tickets.store_id == stores.store_id,'inner').groupBy(tickets.product_id,stores.store_id).count().groupBy('product_id').count().show()

+----------+-----+
|product_id|count|
+----------+-----+
|         7|    6|
|         3|    6|
|         8|    6|
|         5|    6|
|         6|    6|
|         9|    6|
|         1|    6|
|        10|    6|
|         4|    6|
|         2|    6|
+----------+-----+



All 10 products are being sold in all 6 stores


# 2. Calculate the 2nd most selling store for each product as we need a target for advertisement. As the previous one, consider only the stores that are included in the store.csv file.

In [47]:
spark.sql("SELECT product_id, store_id,count(*) as count FROM EMP WHERE store_id IN (SELECT store_id FROM STORE) GROUP BY product_id,store_id").show()

+----------+--------+-----+
|product_id|store_id|count|
+----------+--------+-----+
|         2|      40|    8|
|         3|      42|   14|
|         1|      41|   17|
|         1|      42|   17|
|         7|      44|   10|
|         2|      42|   12|
|         4|      42|   17|
|         2|      41|   14|
|         4|      44|   10|
|        10|      45|   15|
|         5|      40|   11|
|         5|      44|   11|
|         8|      44|    9|
|         4|      40|   22|
|        10|      43|   14|
|         3|      43|   17|
|         3|      45|   14|
|         4|      41|   17|
|         3|      44|   15|
|         5|      45|   10|
+----------+--------+-----+
only showing top 20 rows



Result and number of purchases in SQL

In [46]:
spark.sql("""
SELECT product_id,store_id,count
FROM (
  SELECT product_id, store_id,count(*) AS count,row_number() OVER (
          PARTITION BY product_id ORDER BY count(*) DESC
      ) rank
  FROM EMP
  WHERE store_id IN (SELECT store_id FROM STORE)
  GROUP BY product_id,store_id
)
WHERE rank=2 """).show()

+----------+--------+-----+
|product_id|store_id|count|
+----------+--------+-----+
|         1|      41|   17|
|        10|      43|   14|
|         2|      41|   14|
|         3|      44|   15|
|         4|      42|   17|
|         5|      41|   13|
|         6|      45|   19|
|         7|      40|   18|
|         8|      41|   11|
|         9|      42|   12|
+----------+--------+-----+



In [48]:
second_stores = spark.sql("""
SELECT product_id,store_id,count
FROM (
  SELECT product_id, store_id,count(*) AS count,row_number() OVER (
          PARTITION BY product_id ORDER BY count(*) DESC
      ) rank
  FROM EMP
  WHERE store_id IN (SELECT store_id FROM STORE)
  GROUP BY product_id,store_id
)
WHERE rank=2 """)

# 3. The marketing team wants to group all these second stores by product category, so they can focus on different stores by using the same advertisement approach. As they don’t care about internal id’s, please provide one row per product category_name and include all the stores within that row.

In [88]:
second_stores.createOrReplaceTempView("SECOND_STORE")
products.createOrReplaceTempView("PRODUCTS")

In [93]:
spark.sql("""
SELECT category_name,concat_ws(',',collect_list(store_id)) AS stores
FROM SECOND_STORE
LEFT JOIN PRODUCTS ON SECOND_STORE.product_id = PRODUCTS.product_id
GROUP BY category_name
""").show()

+-------------+--------+
|category_name|  stores|
+-------------+--------+
|       Cereal|42,41,40|
|   Vegetables|   43,42|
|Dairy product|41,41,44|
|      Cookies|45,40,41|
+-------------+--------+



#4. Now, let’s imagine that the integration team is developing a new version for the stores model. They will send this new data for the “version 2” of stores available only for some countries. (That means that you will be receiving both versions simultaneously). They have changed the meaning of store_id and country, so the new store_id has the country prepended to it like “FR99” and the country field is omitted.

In [115]:
stores_v2 = spark.read.csv('stores_v2.csv', header=True, sep=",")

In [96]:
stores_v2.show()

+--------+-------+
|store_id|version|
+--------+-------+
|    DE46|      1|
|    DE47|      2|
+--------+-------+



In [97]:
from pyspark.sql.functions import split

In [116]:
stores_v2 = stores_v2.select(stores_v2.store_id.substr(3,4).alias("store_id"),stores_v2.store_id.substr(1,2).alias("country"),"version")

In [117]:
stores_v2.show()

+--------+-------+-------+
|store_id|country|version|
+--------+-------+-------+
|      46|     DE|      1|
|      47|     DE|      2|
+--------+-------+-------+



Now we just need to concat the 2 versions together

In [118]:
stores.union(stores_v2).show()

+--------+-------+-------+
|store_id|country|version|
+--------+-------+-------+
|      40|     DE|      1|
|      41|     DE|      1|
|      42|     DE|      1|
|      43|     DE|      1|
|      44|     DE|      1|
|      45|     DE|      1|
|      46|     DE|      1|
|      47|     DE|      2|
+--------+-------+-------+



And do the same process as before

5. Considering this ETL complete, we need to productize it: prepare it on the version control system, automatically deploy the changes, ensure that there is no software regression on new releases, and check that the process works fine. Which steps would you take?